# Upload Census data

In [1]:
import boto3
import yaml
import pandas as pd
import io

with open('/home/eouser/Desktop/DEDL/credentials.yaml', 'r') as f:
    credentials = yaml.safe_load(f)


def setup_S3(site_name):
	S3_URL = f"https://s3.{site_name}.data.destination-earth.eu"
	session = boto3.Session(
			aws_access_key_id=credentials[site_name]["key"],
			aws_secret_access_key=credentials[site_name]["secret"],
			region_name=credentials[site_name]["region"],
	)
	return session.client('s3', endpoint_url=S3_URL)



s3 = setup_S3("central")

def move_s3_objects(source_bucket, prefix=''):
    response = s3.list_objects_v2(Bucket=source_bucket, Prefix=prefix)
    if 'Contents' not in response: return print("No files found.")

    objects = []
    bytes_data = []
    for obj in response['Contents']:
        source_key = obj['Key']
        destination_key = source_key.replace(prefix,'',1)
        objects.append(source_key)
        bytes_data.append(s3.get_object(Bucket=source_bucket, Key=source_key)['Body'].read())
        print(source_key)
        # # Copy object to the destination bucket
        # copy_source = {'Bucket': source_bucket, 'Key': source_key}
        # s3.copy_object(CopySource=copy_source, Bucket=destination_bucket, Key=destination_key)
        
        # # Delete object from the source bucket after copying
        # s3.delete_object(Bucket=source_bucket, Key=source_key)
        
        # print(f"Moved {source_key} to {destination_bucket}")
    return dict(zip(objects, bytes_data))

source_bucket = "energy-storage"
destination_bucket = "datadis"
prefix = f"INE/census.tsv"
files = move_s3_objects(source_bucket, prefix)

filename, binary_content = list(files.items())[0]
print(filename)
file_like = io.BytesIO(binary_content)
df_postal = pd.read_csv(file_like, sep="\t", dtype=str)
df_postal

INE/census.tsv
INE/census.tsv


,Municipality code,District code,Section code,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,...,Non-main Homes,Owned Homes,Rental Homes,Homes in another type of tenure regime,Total Households,Households with 1 person,Households with 2 people,Households with 3 people,Households of 4 people,Households of 5 or more people
0,04001,01,001,1260,0.4857,0.5143,49.4994,0.1048,0.6008,0.2944,...,508,394,80,94,568,199,182,90,73,24
1,04002,01,001,1212,0.4719,0.5281,49.4636,0.0891,0.6287,0.2822,...,338,429,89,65,583,241,165,93,67,17
2,04003,01,001,869,0.5086,0.4914,45.0789,0.1208,0.6789,0.2002,...,82,256,29,28,313,69,86,64,57,37
3,04003,01,002,1781,0.5014,0.4986,40.2112,0.1791,0.6912,0.1297,...,163,535,38,53,626,119,156,139,154,58
4,04003,01,003,2235,0.5087,0.4913,38.1048,0.1884,0.7051,0.1065,...,256,701,82,78,861,225,215,187,174,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36328,52001,08,011,2461,0.5055,0.4945,36.5592,0.2206,0.6757,0.1036,...,79,509,123,22,654,65,111,140,160,178
36329,52001,08,012,2361,0.4968,0.5032,41.0614,0.1817,0.6569,0.1614,...,53,618,168,45,831,185,205,179,145,117
36330,52001,08,013,1693,0.4702,0.5298,37.8859,0.1991,0.6964,0.1045,...,139,331,208,41,580,139,149,127,98,67
36331,52001,08,014,1252,0.5088,0.4912,39.4877,0.2069,0.6142,0.1789,...,6,308,58,29,395,69,93,88,58,87


In [2]:
df = df_postal[df_postal['Municipality code'].str[:2].isin(['08','17','43','25'])]

df.loc[:,'CUSEC'] = df.loc[:,'Municipality code'] + df.loc[:,'District code'] + df.loc[:,'Section code']
df.drop(columns=['Municipality code','District code','Section code'], inplace=True)
df[df.columns[:36]] = df[df.columns[:36]].astype(float)


/tmp/ipykernel_2443523/4053062274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'CUSEC'] = df.loc[:,'Municipality code'] + df.loc[:,'District code'] + df.loc[:,'Section code']
/tmp/ipykernel_2443523/4053062274.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Municipality code','District code','Section code'], inplace=True)
/tmp/ipykernel_2443523/4053062274.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
# Load socio-economic data aggregated by municipality code (regular pandas dataframe)

# # Load municipality shapefile (with municipality codes)
gdf= gpd.read_file('/home/eouser/seccionado_2022/España_Seccionado2022_ETRS89H30/SECC_CE_20220101.shp')
gdf = gdf.to_crs(epsg=25831)
gdf.drop(columns=['CUMUN', 'CSEC', 'CDIS', 'CMUN', 'CPRO', 'CCA', 'CUDIS',
       'CLAU2', 'NPRO', 'NCA', 'CNUT0', 'CNUT1', 'CNUT2', 'CNUT3', 'NMUN'],inplace = True)
gdf['area_total'] = gdf['geometry'].area


# # Load postal code shapefile (with postal codes)
gdf_postal = gpd.read_file('/home/eouser/COD_POSTAL_CUSEC.gpkg')
gdf_postal = gdf_postal.to_crs(epsg=25831)
gdf_postal['area_intersected'] = gdf_postal['geometry'].area

In [4]:
columns_to_convert_int = ['Total People','Total Homes', 'Main Homes', 'Non-main Homes', 'Owned Homes',
       'Rental Homes', 'Homes in another type of tenure regime',
       'Total Households', 'Households with 1 person',
       'Households with 2 people', 'Households with 3 people',
       'Households of 4 people', 'Households of 5 or more people'] 

In [5]:
gdf_postal[gdf_postal['CUSEC'] == '4314804002']['area_intersected'].sum()

53702.03029795375

In [6]:
filter_list = ['22487','22571', '22572', '22530', '22449', '22560', '22540', '22589', '22558', '22585', '22520', '22486', '22549', '22590']

merged_df = df.merge(gdf, how='inner',on='CUSEC')
merged_df = merged_df.merge(gdf_postal, how = 'inner',on='CUSEC')

merged_df = merged_df[~merged_df['COD_POSTAL'].isin(filter_list)]


In [7]:
merged_df[merged_df['Main Homes'] != merged_df['Total Households']][['Main Homes', 'Total Households']]

,Main Homes,Total Households


In [8]:
merged_df = merged_df.dropna(how='all').dropna(how='all', axis=1)
nulls = merged_df[merged_df.isnull().any(axis=1)]
nulls

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 2 people,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected
2048,73.0,0.369863,0.630137,52.79,0.034247,0.301370,0.164384,0.000000,0.054795,NaN,...,9.0,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((372600.381 4593846.712, 372250...",2.134911e+04
2049,73.0,0.369863,0.630137,52.79,0.034247,0.301370,0.164384,0.000000,0.054795,NaN,...,9.0,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43420,"MULTIPOLYGON (((367282.621 4599357.322, 367286...",4.859346e+03
2050,73.0,0.369863,0.630137,52.79,0.034247,0.301370,0.164384,0.000000,0.054795,NaN,...,9.0,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43429,"MULTIPOLYGON (((367939.406 4599181.625, 367934...",1.067674e+04
2051,73.0,0.369863,0.630137,52.79,0.034247,0.301370,0.164384,0.000000,0.054795,NaN,...,9.0,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((370845.558 4599387.678, 370954...",3.084079e+07
2052,73.0,0.369863,0.630137,52.79,0.034247,0.301370,0.164384,0.000000,0.054795,NaN,...,9.0,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,08712,"MULTIPOLYGON (((373954.333 4597517.788, 373732...",5.894601e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,79.0,0.379747,0.620253,60.30,0.000000,0.291139,0.208861,0.063291,0.088608,NaN,...,17.0,4.0,2.0,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353329.609 4600340.172, 353646...",5.884012e+04
12951,79.0,0.379747,0.620253,60.30,0.000000,0.291139,0.208861,0.063291,0.088608,NaN,...,17.0,4.0,2.0,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((354850.829 4603222.564, 354876...",5.135378e+03
12952,79.0,0.379747,0.620253,60.30,0.000000,0.291139,0.208861,0.063291,0.088608,NaN,...,17.0,4.0,2.0,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353361.593 4600341.468, 353359...",1.360539e+03
12953,79.0,0.379747,0.620253,60.30,0.000000,0.291139,0.208861,0.063291,0.088608,NaN,...,17.0,4.0,2.0,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43413,"MULTIPOLYGON (((352036.063 4600123.894, 352076...",1.115988e+04


In [9]:

merged_df['proportion'] = (merged_df['area_intersected'] / merged_df['area_total'])
df_orig = merged_df.copy()

In [4]:
gdf_postal

,COD_POSTAL,CUSEC,geometry,area_intersected
0,43001,4314804002,"MULTIPOLYGON (((353133.728 4553199.905, 353130...",3.429901e+03
1,43001,4314804012,"MULTIPOLYGON (((352768.383 4553434.277, 352768...",1.282481e+04
2,43001,4314805001,"MULTIPOLYGON (((353290.401 4553038.605, 353219...",1.505845e+03
3,43001,4314805002,"MULTIPOLYGON (((353138.385 4553132.524, 353206...",5.068206e+04
4,43001,4314805003,"MULTIPOLYGON (((353100.803 4552787.955, 353091...",6.330244e+04
...,...,...,...,...
13249,25718,2500501001,"MULTIPOLYGON (((376742.492 4690365.1, 376699.9...",1.116079e+04
13250,25718,2520301001,"MULTIPOLYGON (((374247.071 4690754.588, 374252...",2.920412e+03
13251,25718,2520301003,"MULTIPOLYGON (((375688.977 4690608.552, 375590...",1.525502e+06
13252,25718,2520301006,"MULTIPOLYGON (((375542.719 4690617.958, 375289...",5.895529e+04


In [10]:

for name, group_df in merged_df.groupby('CUSEC')['proportion']:
    if name[:2] in ['08','17','43','25']:
        sum = group_df.sum()
        if sum != 1:
            group_df = group_df * (1 / sum)
            merged_df.loc[group_df.index, 'proportion'] = group_df

In [11]:
df_orig = merged_df.copy()

In [12]:
merged_df = df_orig.copy()
merged_df.update(merged_df.iloc[:, :3].mul(merged_df['proportion'],0))
merged_df.update(merged_df.iloc[:, 4:36].mul(merged_df['proportion'],0))

In [13]:
group = merged_df[merged_df['CUSEC'] == '4314804002']
group[columns_to_convert_int]

,Total People,Total Homes,Main Homes,Non-main Homes,Owned Homes,Rental Homes,Homes in another type of tenure regime,Total Households,Households with 1 person,Households with 2 people,Households with 3 people,Households of 4 people,Households of 5 or more people
12675,56.651902,35.000273,22.354189,12.646084,13.157037,7.792032,1.405120,22.354189,6.323042,7.089471,4.598576,2.874110,1.468990
12676,49.144103,30.361859,19.391698,10.970161,11.413399,6.759392,1.218907,19.391698,5.485080,6.149939,3.989149,2.493218,1.274312
12677,772.027716,476.968645,304.633259,172.335386,179.298432,106.186450,19.148376,304.633259,86.167693,96.612262,62.667413,39.167133,20.018757
12678,9.176279,5.669223,3.620854,2.048369,2.131131,1.262126,0.227597,3.620854,1.024185,1.148328,0.744861,0.465538,0.237942


In [14]:
nulls = merged_df[merged_df.isnull().any(axis=1)]
nulls

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
2048,0.050195,0.000254,0.000433,52.79,0.000024,0.000207,0.000113,0.000000,0.000038,NaN,...,0.004126,0.003438,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((372600.381 4593846.712, 372250...",2.134911e+04,0.000688
2049,0.011425,0.000058,0.000099,52.79,0.000005,0.000047,0.000026,0.000000,0.000009,NaN,...,0.000939,0.000783,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43420,"MULTIPOLYGON (((367282.621 4599357.322, 367286...",4.859346e+03,0.000157
2050,0.025103,0.000127,0.000217,52.79,0.000012,0.000104,0.000057,0.000000,0.000019,NaN,...,0.002063,0.001719,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43429,"MULTIPOLYGON (((367939.406 4599181.625, 367934...",1.067674e+04,0.000344
2051,72.511918,0.367390,0.625924,52.79,0.034018,0.299355,0.163284,0.000000,0.054428,NaN,...,5.959884,4.966570,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((370845.558 4599387.678, 370954...",3.084079e+07,0.993314
2052,0.138592,0.000702,0.001196,52.79,0.000065,0.000572,0.000312,0.000000,0.000104,NaN,...,0.011391,0.009493,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,08712,"MULTIPOLYGON (((373954.333 4597517.788, 373732...",5.894601e+04,0.001899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12950,0.426849,0.002052,0.003351,60.30,0.000000,0.001573,0.001129,0.000342,0.000479,NaN,...,0.021613,0.010806,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353329.609 4600340.172, 353646...",5.884012e+04,0.005403
12951,0.037254,0.000179,0.000292,60.30,0.000000,0.000137,0.000098,0.000030,0.000042,NaN,...,0.001886,0.000943,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((354850.829 4603222.564, 354876...",5.135378e+03,0.000472
12952,0.009870,0.000047,0.000077,60.30,0.000000,0.000036,0.000026,0.000008,0.000011,NaN,...,0.000500,0.000250,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353361.593 4600341.468, 353359...",1.360539e+03,0.000125
12953,0.080958,0.000389,0.000636,60.30,0.000000,0.000298,0.000214,0.000065,0.000091,NaN,...,0.004099,0.002050,0.0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43413,"MULTIPOLYGON (((352036.063 4600123.894, 352076...",1.115988e+04,0.001025


Converting columns to int

In [15]:
def rounding2int(group):
    for column in columns_to_convert_int:
        values = group[column]
        if values.isnull().all():
            group[column] = values
            continue
        rounded_vals = np.floor(values).astype(int)
        residual = int(round(values.sum()) - rounded_vals.sum())
        diffs = values - rounded_vals
        sorted_indexes = np.argsort(-diffs)
        sorted_indexes = values.index[sorted_indexes]
        for i in range(residual):
            rounded_vals[sorted_indexes[i]] += 1
            
        group[column] = rounded_vals
    return group



In [16]:
merged_df = merged_df.groupby('CUSEC').apply(rounding2int).reset_index(drop=True)
merged_df

/tmp/ipykernel_2443523/1804337115.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('CUSEC').apply(rounding2int).reset_index(drop=True)


,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
0,1497,0.501090,0.497694,44.8191,0.147221,0.618348,0.233216,0.093286,0.137932,0.185175,...,133,85,47,0800101001,"POLYGON ((409003.472 4595032.676, 408925.52 45...",9.484034e+05,08630,"MULTIPOLYGON (((408895.607 4595056.978, 408858...",9.472508e+05,0.998785
1,2,0.000610,0.000606,44.8191,0.000179,0.000752,0.000284,0.000114,0.000168,0.000225,...,0,0,0,0800101001,"POLYGON ((409003.472 4595032.676, 408925.52 45...",9.484034e+05,08760,"MULTIPOLYGON (((408895.607 4595056.978, 408919...",1.152662e+03,0.001215
2,1846,0.497300,0.502700,42.8167,0.176100,0.625100,0.198800,0.104000,0.146300,0.188700,...,147,128,57,0800101002,"POLYGON ((408518.067 4597223.24, 408527.973 45...",1.270398e+05,08630,"MULTIPOLYGON (((408527.973 4597107.661, 408537...",1.270398e+05,1.000000
3,848,0.507059,0.491679,40.8853,0.197650,0.662263,0.138825,0.035255,0.065917,0.197950,...,70,87,23,0800101003,"POLYGON ((410255.478 4594336.179, 410249.335 4...",2.498596e+06,08630,"MULTIPOLYGON (((409874.313 4594700.607, 409818...",2.495443e+06,0.998738
4,1,0.000641,0.000621,40.8853,0.000250,0.000837,0.000175,0.000045,0.000083,0.000250,...,0,0,0,0800101003,"POLYGON ((410255.478 4594336.179, 410249.335 4...",2.498596e+06,08760,"MULTIPOLYGON (((409874.313 4594700.607, 409900...",3.153298e+03,0.001262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13207,5,0.001460,0.001433,39.3104,0.000575,0.001903,0.000416,0.000245,0.000320,0.000151,...,1,0,0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43481,"MULTIPOLYGON (((347764.729 4551520.952, 347916...",1.232147e+04,0.002893
13208,245,0.071532,0.070228,39.3104,0.028168,0.093222,0.020371,0.011993,0.015693,0.007386,...,20,19,9,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43110,"MULTIPOLYGON (((347788.89 4553021.763, 347788....",6.037450e+05,0.141761
13209,13,0.003990,0.003917,39.3104,0.001571,0.005200,0.001136,0.000669,0.000875,0.000412,...,1,1,0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43100,"MULTIPOLYGON (((348443.518 4553014.726, 348178...",3.367690e+04,0.007907
13210,1460,0.426798,0.419017,39.3104,0.168063,0.556208,0.121544,0.071556,0.093632,0.044067,...,120,113,51,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43006,"MULTIPOLYGON (((347918.265 4551567.884, 347916...",3.602244e+06,0.845815


In [17]:
merged_dfnulls = merged_df[merged_df.isnull().any(axis=1)]
merged_dfnulls

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
2048,0,0.000254,0.000433,52.79,0.000024,0.000207,0.000113,0.000000,0.000038,NaN,...,0,0,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((372600.381 4593846.712, 372250...",2.134911e+04,0.000688
2049,0,0.000058,0.000099,52.79,0.000005,0.000047,0.000026,0.000000,0.000009,NaN,...,0,0,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43420,"MULTIPOLYGON (((367282.621 4599357.322, 367286...",4.859346e+03,0.000157
2050,0,0.000127,0.000217,52.79,0.000012,0.000104,0.000057,0.000000,0.000019,NaN,...,0,0,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43429,"MULTIPOLYGON (((367939.406 4599181.625, 367934...",1.067674e+04,0.000344
2051,73,0.367390,0.625924,52.79,0.034018,0.299355,0.163284,0.000000,0.054428,NaN,...,6,5,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((370845.558 4599387.678, 370954...",3.084079e+07,0.993314
2052,0,0.000702,0.001196,52.79,0.000065,0.000572,0.000312,0.000000,0.000104,NaN,...,0,0,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,08712,"MULTIPOLYGON (((373954.333 4597517.788, 373732...",5.894601e+04,0.001899
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,1,0.002052,0.003351,60.30,0.000000,0.001573,0.001129,0.000342,0.000479,NaN,...,0,0,0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353329.609 4600340.172, 353646...",5.884012e+04,0.005403
12931,0,0.000179,0.000292,60.30,0.000000,0.000137,0.000098,0.000030,0.000042,NaN,...,0,0,0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((354850.829 4603222.564, 354876...",5.135378e+03,0.000472
12932,0,0.000047,0.000077,60.30,0.000000,0.000036,0.000026,0.000008,0.000011,NaN,...,0,0,0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43427,"MULTIPOLYGON (((353361.593 4600341.468, 353359...",1.360539e+03,0.000125
12933,0,0.000389,0.000636,60.30,0.000000,0.000298,0.000214,0.000065,0.000091,NaN,...,0,0,0,4315901001,"POLYGON ((355029.037 4603058.322, 355019.037 4...",1.088995e+07,43413,"MULTIPOLYGON (((352036.063 4600123.894, 352076...",1.115988e+04,0.001025


In [18]:
merged_df = merged_df[~(merged_df.isnull().any(axis=1) & (merged_df['Total People'] < 2) & (merged_df['Non-main Homes'] == 0) & (merged_df['Main Homes'] == 0))]



In [19]:
merged_df[merged_df['Total People'] == 0]

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
7,0,6.279225e-05,6.493812e-05,39.3888,2.652960e-05,8.441701e-05,1.679654e-05,5.684002e-06,9.924650e-06,2.598036e-05,...,0,0,0,0800101004,"POLYGON ((408060.875 4597266.757, 408057.685 4...",6.240690e+06,08760,"MULTIPOLYGON (((408707.865 4595046.907, 408731...",797.125628,1.277304e-04
17,0,1.380296e-04,1.611891e-04,43.2562,6.322490e-05,1.780650e-04,5.795865e-05,6.313513e-06,9.485230e-06,2.537374e-05,...,0,0,0,0800201001,"POLYGON ((386094.215 4624712.632, 386126.647 4...",4.311809e+07,08719,"MULTIPOLYGON (((383776.344 4616591.456, 383850...",12901.736677,2.992186e-04
20,0,4.440919e-04,5.186047e-04,43.2562,2.034178e-04,5.729008e-04,1.864743e-04,2.031290e-05,3.051748e-05,8.163667e-05,...,0,0,0,0800201001,"POLYGON ((386094.215 4624712.632, 386126.647 4...",4.311809e+07,08281,"MULTIPOLYGON (((382675.351 4621015.862, 382683...",41509.642430,9.626966e-04
21,0,1.564391e-04,1.826875e-04,43.2562,7.165745e-05,2.018142e-04,6.568882e-05,7.155571e-06,1.075031e-05,2.875794e-05,...,0,0,0,0800201001,"POLYGON ((386094.215 4624712.632, 386126.647 4...",4.311809e+07,08263,"MULTIPOLYGON (((386064.431 4624713.994, 386025...",14622.492554,3.391266e-04
22,0,5.092673e-04,5.947155e-04,43.2562,2.332716e-04,6.569802e-04,2.138415e-04,2.329404e-05,3.499626e-05,9.361774e-05,...,0,0,0,0800201001,"POLYGON ((386094.215 4624712.632, 386126.647 4...",4.311809e+07,08259,"MULTIPOLYGON (((387634.27 4623049.016, 387466....",47601.634294,1.103983e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13163,0,4.588727e-05,4.494264e-05,45.2577,1.305226e-05,5.658704e-05,2.119062e-05,8.519846e-06,9.891378e-06,2.852059e-06,...,0,0,0,4390201002,"POLYGON ((303832.837 4510238.337, 303848.116 4...",9.649152e+06,43870,"MULTIPOLYGON (((307372.709 4505900.057, 307397...",876.433166,9.082992e-05
13165,0,6.796402e-05,7.306943e-05,45.3178,2.088706e-05,8.824463e-05,3.190177e-05,2.630274e-05,2.541423e-05,5.443891e-06,...,0,0,0,4390301001,"POLYGON ((301214.354 4521614.303, 301214.35 45...",2.390093e+07,43500,"MULTIPOLYGON (((299388.458 4520774.477, 299113...",3370.832061,1.410335e-04
13168,0,4.979486e-05,5.353542e-05,45.3178,1.530321e-05,6.465376e-05,2.337331e-05,1.927110e-05,1.862012e-05,3.988549e-06,...,0,0,0,4390301001,"POLYGON ((301214.354 4521614.303, 301214.35 45...",2.390093e+07,43896,"MULTIPOLYGON (((298824.87 4520064.07, 298834.8...",2469.690697,1.033303e-04
13176,0,5.305082e-05,6.035409e-05,43.8060,1.854170e-05,7.090075e-05,2.396246e-05,2.238613e-05,2.358822e-05,3.810405e-06,...,0,0,0,4390401001,"POLYGON ((300635.476 4514704.717, 300636.896 4...",1.731525e+07,43897,"MULTIPOLYGON (((296777.879 4514178.191, 296609...",1963.634204,1.134049e-04


In [20]:
percentage_columns = merged_df.columns[merged_df.columns.str.startswith('Percentage')]

In [21]:
nulls = merged_df[merged_df.isnull().any(axis=1)]
nulls

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
2051,73,0.367390,0.625924,52.79,0.034018,0.299355,0.163284,0.000000,0.054428,NaN,...,6,5,0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((370845.558 4599387.678, 370954...",3.084079e+07,0.993314
2280,84,0.519274,0.472067,43.11,0.106215,0.306844,0.082612,0.000000,0.023603,NaN,...,6,7,2,0804501001,"POLYGON ((400729.22 4662107.948, 400829.519 46...",3.429357e+07,08617,"MULTIPOLYGON (((392153.123 4660226.124, 392441...",3.399663e+07,0.991341
2450,72,0.440278,0.550347,47.04,0.055035,0.350846,0.089431,0.013759,0.027517,NaN,...,2,2,4,0805701001,"POLYGON ((414598.09 4672398.036, 414549.565 46...",2.478975e+07,08604,"MULTIPOLYGON (((411271.642 4672724.052, 411274...",2.455733e+07,0.990624
2800,44,0.383196,0.608605,53.47,0.033811,0.338114,0.123975,0.090164,0.112705,NaN,...,4,2,0,0808001001,"POLYGON ((404588.713 4671509.272, 404588.733 4...",2.965122e+07,08698,"MULTIPOLYGON (((395962.166 4671452.358, 395984...",2.940810e+07,0.991801
2959,27,0.330809,0.661618,52.08,0.036757,0.330809,0.128648,0.000000,0.000000,NaN,...,0,1,1,0809301001,"POLYGON ((399401.264 4682850.774, 399441.197 4...",3.657601e+07,08695,"MULTIPOLYGON (((397583.599 4679482.879, 397463...",3.629902e+07,0.992427
2971,1,0.002389,0.004379,51.28,0.000199,0.002508,0.000677,0.000239,0.000557,NaN,...,0,0,0,0809501001,"POLYGON ((426026.766 4618762.775, 425775.875 4...",2.384902e+07,08183,"MULTIPOLYGON (((425996.974 4618763.974, 425982...",1.613980e+05,0.006767
2972,84,0.349191,0.640184,51.28,0.029099,0.366651,0.098937,0.034919,0.081478,NaN,...,4,6,1,0809501001,"POLYGON ((426026.766 4618762.775, 425775.875 4...",2.384902e+07,08183,"MULTIPOLYGON (((423420.04 4621680.262, 423370....",2.359561e+07,0.989375
4558,49,0.425773,0.567697,45.95,0.091237,0.253436,0.152062,0.141924,0.141924,NaN,...,2,4,2,0817701001,"POLYGON ((414389.868 4664646.857, 414534.325 4...",3.771889e+07,08606,"MULTIPOLYGON (((412580.536 4660232.177, 412535...",3.747259e+07,0.993470
5063,91,0.483190,0.504665,47.76,0.064425,0.311389,0.118113,0.000000,0.010738,NaN,...,4,7,3,0819501001,"POLYGON ((430597.433 4658963.612, 430391.892 4...",1.325269e+07,08586,"MULTIPOLYGON (((429729.763 4659379.909, 429597...",1.309175e+07,0.987856
5480,28,0.388863,0.600969,48.70,0.053027,0.247458,0.194431,0.035351,0.035351,NaN,...,2,0,1,0821601001,"POLYGON ((421895.7 4674583.337, 421880.38 4674...",2.136616e+07,08607,"MULTIPOLYGON (((421850.807 4674452.584, 421776...",2.114891e+07,0.989832


In [22]:
from sklearn.impute import KNNImputer

numerical_float_columns = merged_df.select_dtypes(include=['int','float']).columns.to_list()

knn_imputer=KNNImputer(n_neighbors=5)
merged_df[numerical_float_columns] = knn_imputer.fit_transform(merged_df[numerical_float_columns])
merged_df

/tmp/ipykernel_2443523/963800920.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df[numerical_float_columns] = knn_imputer.fit_transform(merged_df[numerical_float_columns])


,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
0,1497.0,0.501090,0.497694,44.8191,0.147221,0.618348,0.233216,0.093286,0.137932,0.185175,...,133.0,85.0,47.0,0800101001,"POLYGON ((409003.472 4595032.676, 408925.52 45...",9.484034e+05,08630,"MULTIPOLYGON (((408895.607 4595056.978, 408858...",9.472508e+05,0.998785
1,2.0,0.000610,0.000606,44.8191,0.000179,0.000752,0.000284,0.000114,0.000168,0.000225,...,0.0,0.0,0.0,0800101001,"POLYGON ((409003.472 4595032.676, 408925.52 45...",9.484034e+05,08760,"MULTIPOLYGON (((408895.607 4595056.978, 408919...",1.152662e+03,0.001215
2,1846.0,0.497300,0.502700,42.8167,0.176100,0.625100,0.198800,0.104000,0.146300,0.188700,...,147.0,128.0,57.0,0800101002,"POLYGON ((408518.067 4597223.24, 408527.973 45...",1.270398e+05,08630,"MULTIPOLYGON (((408527.973 4597107.661, 408537...",1.270398e+05,1.000000
3,848.0,0.507059,0.491679,40.8853,0.197650,0.662263,0.138825,0.035255,0.065917,0.197950,...,70.0,87.0,23.0,0800101003,"POLYGON ((410255.478 4594336.179, 410249.335 4...",2.498596e+06,08630,"MULTIPOLYGON (((409874.313 4594700.607, 409818...",2.495443e+06,0.998738
4,1.0,0.000641,0.000621,40.8853,0.000250,0.000837,0.000175,0.000045,0.000083,0.000250,...,0.0,0.0,0.0,0800101003,"POLYGON ((410255.478 4594336.179, 410249.335 4...",2.498596e+06,08760,"MULTIPOLYGON (((409874.313 4594700.607, 409900...",3.153298e+03,0.001262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13207,5.0,0.001460,0.001433,39.3104,0.000575,0.001903,0.000416,0.000245,0.000320,0.000151,...,1.0,0.0,0.0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43481,"MULTIPOLYGON (((347764.729 4551520.952, 347916...",1.232147e+04,0.002893
13208,245.0,0.071532,0.070228,39.3104,0.028168,0.093222,0.020371,0.011993,0.015693,0.007386,...,20.0,19.0,9.0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43110,"MULTIPOLYGON (((347788.89 4553021.763, 347788....",6.037450e+05,0.141761
13209,13.0,0.003990,0.003917,39.3104,0.001571,0.005200,0.001136,0.000669,0.000875,0.000412,...,1.0,1.0,0.0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43100,"MULTIPOLYGON (((348443.518 4553014.726, 348178...",3.367690e+04,0.007907
13210,1460.0,0.426798,0.419017,39.3104,0.168063,0.556208,0.121544,0.071556,0.093632,0.044067,...,120.0,113.0,51.0,4390701004,"POLYGON ((348014.206 4553073.556, 348378.495 4...",4.258902e+06,43006,"MULTIPOLYGON (((347918.265 4551567.884, 347916...",3.602244e+06,0.845815


In [23]:
 merged_df[merged_df.isnull().any(axis=1)]

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion


In [24]:
merged_df[merged_df['CUSEC'] == '0802101001']

,Total People,Percentage of women,Percentage of men,Average age,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,...,Households with 3 people,Households of 4 people,Households of 5 or more people,CUSEC,geometry_x,area_total,COD_POSTAL,geometry_y,area_intersected,proportion
2051,73.0,0.36739,0.625924,52.79,0.034018,0.299355,0.163284,0.0,0.054428,0.039556,...,6.0,5.0,0.0,0802101001,"POLYGON ((370615.939 4599540.447, 370875.458 4...",3.104841e+07,43421,"MULTIPOLYGON (((370845.558 4599387.678, 370954...",3.084079e+07,0.993314


In [25]:
def aggregate_census(group):
    result = {}
    for column in percentage_columns:
        count = (group['Total People'] * group[column]).sum()
        ppl = group['Total People'].sum()
        result[column] =  count / ppl if ppl != 0 else 0

    for column in columns_to_convert_int:
        result[column] = group[column].sum()
    result['Average age'] = group['Average age'].mean()
    return pd.Series(result)

merged_df = merged_df.groupby('COD_POSTAL').apply(aggregate_census).reset_index()


/tmp/ipykernel_2443523/2248077462.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merged_df = merged_df.groupby('COD_POSTAL').apply(aggregate_census).reset_index()


In [26]:
merged_df[merged_df['COD_POSTAL'] == '25217']

,COD_POSTAL,Percentage of women,Percentage of men,Percentage of people under 16 years,Percentage of people between 16 (inclusive) and 64 (inclusive) years,Percentage of people over 64 years of age,Percentage foreigners,Percentage of people born abroad,Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over,Percentage of people pursuing university studies ( escur = 09 10 11 12) on population aged 16 and over,...,Owned Homes,Rental Homes,Homes in another type of tenure regime,Total Households,Households with 1 person,Households with 2 people,Households with 3 people,Households of 4 people,Households of 5 or more people,Average age
716,25217,0.34782,0.409506,0.09006,0.440352,0.226946,0.035929,0.04832,0.029072,0.021868,...,120.0,39.0,32.0,190.0,56.0,55.0,37.0,23.0,19.0,49.555111


In [39]:
merged_df[columns_to_convert_int] = merged_df[columns_to_convert_int].astype('int64')
merged_df['Average age'] = merged_df['Average age'].astype('int64')

In [47]:
merged_df.columns

Index(['COD_POSTAL', 'Percentage of women', 'Percentage of men',
       'Percentage of people under 16 years',
       'Percentage of people between 16 (inclusive) and 64 (inclusive) years',
       'Percentage of people over 64 years of age', 'Percentage foreigners',
       'Percentage of people born abroad',
       'Percentage of people pursuing higher education (escur =08 09 10 11 12 ) of the population aged 16 and over',
       'Percentage of people pursuing university studies ( escur = 09 10 11 12) on population aged 16 and over',
       'Percentage of people with higher education (esreal_cneda=08 09 10 11 12) on population aged 16 and over',
       'Percentage of population unemployment over active population= Unemployed /Active',
       'Percentage of employed population over population aged 16 and over =Employed/ Population 16 and +',
       'Percentage of active population over population aged 16 and over= Active / Population 16 and +',
       'Percentage of disability pensioner

Upload dataframe

In [50]:
import psycopg2
import polars as pl
import re
from tqdm import tqdm

with open('/home/eouser/Desktop/DEDL/credentials.yaml', 'r') as f:
    c = yaml.safe_load(f)["postgres"]
    
conn = psycopg2.connect(f"dbname={c['db_name']} user={c['db_user']} password={c['db_password']} host={c['db_host']} port={c['db_port']} sslmode=require")
cursor = conn.cursor()


dtype_map = {
    'object': 'VARCHAR',
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'datetime64[ns]': 'TIMESTAMP',
    'datetime64[us]': 'TIMESTAMP'
}
def check_table_exists(table_name):
    query = f"""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = 'public' AND table_name = '{table_name}'
    """
    cursor.execute(query)
    return cursor.fetchone()[0] 
    
def upload_data(df, table_name):
    if isinstance(df, pl.DataFrame):
        df = df.to_pandas()
    print(len(df))
            
    if not check_table_exists(table_name):
        columns = []
        for col_name, dtype in df.dtypes.items():
            pg_type = dtype_map.get(str(dtype), 'VARCHAR')
            name = re.sub(r'[^\w\s]', '', col_name).replace(' ', '_')
            columns.append(f"{name} {pg_type}")
        create_table_query = f"""
        CREATE TABLE {table_name} (
            {', '.join(columns)},
            PRIMARY KEY (COD_POSTAL)
        );
        """
        cursor.execute(create_table_query)
        conn.commit()
        
    name = [re.sub(r'[^\w\s]', '',x).replace(' ', '_') for x in df.columns]
    insert_query = f"""
    INSERT INTO {table_name} ({','.join(name)})
    VALUES ({','.join(['%s'] * len(df.columns))});
    """

    for x in tqdm(df.to_numpy(), desc=f"Inserting {table_name} rows", unit="rows"):
        cursor.execute(insert_query,tuple(x))
    conn.commit()
    
upload_data(merged_df, 'census2021')


1137


Inserting census2021 rows:   0%|          | 0/1137 [00:00<?, ?rows/s]

Inserting census2021 rows: 100%|██████████| 1137/1137 [00:00<00:00, 2464.50rows/s]
